Set up requests from Open-Meteo API. <br>
<br>
Sample API Endpoint for latitude=52.52 and longitude=13.41<br>
Selects 15 min interval data for wind_speed_80m, wind_direction_80m, wind_gusts_10m from dates 2025-12-01 to 2025-12-30<br> 
https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&minutely_15=wind_speed_80m,wind_direction_80m,wind_gusts_10m&start_date=2025-12-01&end_date=2025-12-30

Start with hourly instead of 15 minute data for a start. Allows faster loading, data is clearer, and more realistic for energy analytics.

In [ ]:
import requests
import pandas as pd

def get_wind_data(lat, lon, date_start, date_end):
    """
    Fetch historical + forecast wind data from Open-Meteo.

    Parameters
    ----------
    lat : float
        Latitude
    lon : float
        Longitude
    date_start : str
        Start date in YYYY-MM-DD format
    date_end : str
        End date in YYYY-MM-DD format

    Returns
    -------
    pd.DataFrame
        DataFrame with timestamp, wind speed, wind direction, and data type
    """

    url = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": lat,
        "longitude": lon,
        # Hourly is usually better for dashboards than 15-min to start
        "hourly": [
            "wind_speed_80m",
            "wind_direction_80m"
        ],
        "wind_speed_unit": "ms",
        "start_date": date_start,
        "end_date": date_end,
        "timezone": "UTC"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()  # fail fast if API errors

    data = response.json()

    # Parse into DataFrame
    df = pd.DataFrame({
        "timestamp": pd.to_datetime(data["hourly"]["time"], utc=True),
        "wind_speed_mps": data["hourly"]["wind_speed_80m"],
        "wind_direction_deg": data["hourly"]["wind_direction_80m"],
    })

    return df



In [8]:
from datetime import datetime, timezone

def add_data_type(df):
    now = datetime.now(timezone.utc)    
    df["data_type"] = df["timestamp"].apply(
        lambda t: "historical" if t <= now else "forecast"
    )
    return df


In [9]:
# Example
lat=52.52 
lon=13.41
date_start="2025-12-01"
date_end="2025-12-10"

df = get_wind_data(lat, lon, date_start, date_end)
df = add_data_type(df)


In [11]:
print(df["timestamp"].dt.tz)
print(type(df["timestamp"].iloc[0]))


UTC
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
import numpy as np

def wind_to_power(
    wind_speed_mps,
    cut_in=3.0,
    rated_speed=12.0,
    cut_out=25.0,
    rated_power_mw=3.0
):
    """
    Convert wind speed to power output using an idealized power curve.

    Parameters
    ----------
    wind_speed_mps : float or np.ndarray
        Wind speed in m/s
    cut_in : float
        Cut-in wind speed (m/s)
    rated_speed : float
        Rated wind speed (m/s)
    cut_out : float
        Cut-out wind speed (m/s)
    rated_power_mw : float
        Rated power (MW)

    Returns
    -------
    power_mw : float or np.ndarray
        Power output (MW)
    """

    ws = np.array(wind_speed_mps)

    power = np.zeros_like(ws, dtype=float)

    # Linear ramp region
    ramp_mask = (ws >= cut_in) & (ws < rated_speed)
    power[ramp_mask] = rated_power_mw * (
        (ws[ramp_mask] - cut_in) / (rated_speed - cut_in)
    )

    # Rated region
    rated_mask = (ws >= rated_speed) & (ws <= cut_out)
    power[rated_mask] = rated_power_mw

    # Outside operating range stays 0
    return power


In [13]:
def add_power_output(df, turbine_params):
    df = df.copy()
    df["power_mw"] = wind_to_power(
        df["wind_speed_mps"].values,
        cut_in=turbine_params["cut_in"],
        rated_speed=turbine_params["rated_speed"],
        cut_out=turbine_params["cut_out"],
        rated_power_mw=turbine_params["rated_power_mw"],
    )
    return df


In [ ]:
# pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 4.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: narwhals
    Found existing installation: narwhals 1.26.0
    Uninstalling narwhals-1.26.0:
      Successfully uninstalled narwhals-1.26.0
Note: you may need to restart the kernel to use updated packages.
